---
title: Big Number Conversion
skip-execution: true
---

In [ ]:
from inspect import getsource
from math import floor, log2

import numpy as np
from ipywidgets import interact

%reload_ext divewidgets

In [ ]:
if not input('Load JupyterAI? [Y/n]').lower()=='n':
    %reload_ext jupyter_ai

An `int` can be arbitrarily large because it is represented similarly to a string of digits. More precisely, as shown in the [source code][PyLong], an integer consists of a collection of fixed-length unsigned integers to store the digits, along with another unsigned integer to store the number of digits, sign and other flags.[^max-int] Integer operations such as subtraction often has to iterate through all the digits, as shown in the [code][x_sub].[^2s-complement]

[PyLong]: https://github.com/python/cpython/blob/6810928927e4d12d9a5dd90e672afb096882b730/Include/cpython/longintrepr.h#L64
[MAX_LONG_DIGITS]: https://github.com/python/cpython/blob/6810928927e4d12d9a5dd90e672afb096882b730/Objects/longobject.c#L159
[x_sub]: https://github.com/python/cpython/blob/6810928927e4d12d9a5dd90e672afb096882b730/Objects/longobject.c#L3696
[a+(-b)]: https://github.com/python/cpython/blob/6810928927e4d12d9a5dd90e672afb096882b730/Objects/longobject.c#L3775
[^max-int]: The reality is that `int` still has a maximum. This is because the number of digits is stored using an unsigned integer of fixed size! (See the [source code][MAX_LONG_DIGITS].)
[^2s-complement]: Note that the code implements the subtraction $a-b$ for non-negative integers $a$ and $b$ directly. It does not resort to addition $a+(-b)$ using the 2's complement representation of $(-b)$. In fact, the addition $a+(-b)$ is actually converted to $a-b$ as shown in [here in the code][a+(-b)].

Similarly in this notebook, we will use iterations to convert non-negative integers with arbitrary size.

## Conversion to Decimal

### Binary-to-Decimal

In a previous lab, we considered converting a byte string to decimal. What about converting a binary string of arbitrary length to decimal?

The simplest way is to use `int` constructor:

In [ ]:
int("0" * 4 + "1" * 4, base=2)

**But how is `int` constructor implemented?**

::::{prf:definition} Binary numbers
:label: def:binary

Given a binary sequence of an arbitrarily length $k$,

$$ 
b_{k-1}\circ \dots \circ b_1\circ b_0,
$$

where $b_i\in \{0,1\}$ for $i\in \{0, \dots, k-1\}$, the decimal number is given by the formula

$$
2^0 \cdot b_0 + 2^1 \cdot b_1 + \dots + 2^{k-1} \cdot b_{k-1}.
$$ (eq:b2d:1)

::::

In mathematics, we use the [summation](https://en.wikipedia.org/wiki/Summation) notation to write the above formula {eq}`eq:b2d:1` concisely as

$$ 
\sum_{i=0}^{k-1} 2^i \cdot b_i,
$$ (eq:b2d:2)

i.e., the sum of $2^i \cdot b_i$ for $i$ taking an integer value from $0$ to $k-1$.

This can be implemented using a `for` loop:

In [ ]:
def binary_to_decimal_v1(binary_str):
    k = len(binary_str)
    decimal = 0  # initialization
    for i in range(k):  # iteration
        decimal += 2**i * (1 if binary_str[(k - 1) - i] == "1" else 0)
    return decimal


binary_to_decimal_v1("0" * 4 + "1" * 4)

::::{attention}
In the above implementation, 

```python
1 if binary_str[(k - 1) - i] == "1" else 0
```

is to avoid using `int` as in `int(binary_str[(k - 1) - i])`.
::::

Note that $b_i$ is given by `binary_str[(k-1)-i]` for different index `i` as shown below:

$$
\begin{array}{c|c:c:c:c|}
\texttt{binary\_str} & b_{k-1} & b_{k-2} & \dots & b_0\\ \text{indexing} & [0] & [1] & \dots & [k-1] 
\end{array}
$$

The following is another way to write the `for` loop.

In [ ]:
def binary_to_decimal_v2(binary_str):
    decimal = 0  # initialization
    for bit in binary_str:
        decimal = decimal * 2 + (1 if bit == "1" else 0)  # iteration
    return decimal


binary_to_decimal_v2("0" * 4 + "1" * 4)

The algorithm implements the same formula factorized as follows:

$$
\begin{aligned} \sum_{i=0}^{k-1} 2^i \cdot b_{i} 
&=  \left(\sum_{i=1}^{k-1} 2^i \cdot b_{i}\right) + b_0\\
&=  \left(\sum_{i=1}^{k-1} 2^{i-1} \cdot b_{i}\right)\times 2 + b_0 \\
&=  \left(\sum_{j=0}^{k-2} 2^{j} \cdot b_{j+1}\right)\times 2 + b_0 && \text{with $j=i-1$} \\
&= \underbrace{(\dots (\underbrace{(\underbrace{\overbrace{0}^{\text{initialization}\kern-2em}\times 2 + b_{k-1}}_{\text{first iteration} }) \times 2 + b_{k-2}}_{\text{second iteration} }) \dots )\times 2 + b_0}_{\text{last iteration} }.\end{aligned}
$$ (eq:b2d:3)



::::{exercise} efficiency
:label: ex:efficiency

Explain whether `binary_to_decimal_v1` or `binary_to_decimal_v2` runs faster?

::::

YOUR ANSWER HERE

::::{exercise} binary-to-decimal conversion
:label: ex:binary-to-decimal 

Write your own converter `binary_to_decimal` below using a `while` loop instead of a `for` loop. 

:::{important}

Your code should contain `while` but NOT `for` nor `int`.
:::

::::

In [ ]:
def binary_to_decimal(binary_str):
    k = len(binary_str)
    decimal = 0
    # YOUR CODE HERE
    raise NotImplementedError
    return decimal

In [ ]:
# test validity
assert getsource(binary_to_decimal).find("for") < 0  # no for
assert getsource(binary_to_decimal).find("int") < 0  # no int
assert getsource(binary_to_decimal).find("while") > 0  # use while

# tests
def test_binary_to_decimal(decimal, binary_str):
    decimal_ = binary_to_decimal(binary_str)
    correct = isinstance(decimal_, int) and decimal_ == decimal
    if not correct:
        print(f"{binary_str} should give {decimal} not {decimal_}.")
    assert correct


test_binary_to_decimal(0, "0")
test_binary_to_decimal(255, "11111111")
test_binary_to_decimal(52154, "1100101110111010")
test_binary_to_decimal(3430, "110101100110")

In [ ]:
# hidden tests

In [ ]:
# binary-to-decimal converter
from ipywidgets import interact

bits = ["0", "1"]


@interact(binary_str="1011")
def convert_byte_to_decimal(binary_str):
    for bit in binary_str:
        if bit not in bits:
            print("Not a binary string.")
            break
    else:
        print("decimal:", binary_to_decimal(binary_str))

### Undecimal-to-Decimal

A base-11 number system is called an [undecimal system](https://en.wikipedia.org/wiki/Undecimal). The digits range from 0 to 10 with 10 denoted as X:

$$
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, X.
$$

For example, the [International Standard Book Number (ISBN)](https://en.wikipedia.org/wiki/International_Standard_Book_Number) uses an undecimal digit.

::::{exercise} undecimal-to-decimal conversion
:label: ex:undecimal-to-decimal

In the following code, assign to `decimal` the integer represented by an undecimal string of arbitrary length.

:::{important}

Your code should NOT contain `int`. You may assume `undecimal_str` is a valid undecimal string consisting of characters from "0123456789X".
:::

::::

In [ ]:
def undecimal_to_decimal(undecimal_str):
    # YOUR CODE HERE
    raise NotImplementedError
    return decimal

In [ ]:
# test validity
assert getsource(undecimal_to_decimal).find("int") < 0  # no int

# tests
def test_undecimal_to_decimal(decimal, undecimal_str):
    decimal_ = undecimal_to_decimal(undecimal_str)
    correct = isinstance(decimal_, int) and decimal_ == decimal
    if not correct:
        print(f"{undecimal_str} should give {decimal} not {decimal_}.")
    assert correct


test_undecimal_to_decimal(27558279079916281, "6662X0X584839464")
test_undecimal_to_decimal(23022771839270, "73769X2556695")
test_undecimal_to_decimal(161804347284488, "476129248X2067")

In [ ]:
# hidden tests

In [ ]:
# undecimal-to-decimal calculator
undecimal_digits = [str(i) for i in range(10)] + ["X"]


@interact(undecimal="X")
def convert_undecimal_to_decimal(undecimal):
    for digit in undecimal:
        if digit not in undecimal_digits:
            print("Not an undecimal string.")
            break
    else:
        print("decimal:", undecimal_to_decimal(undecimal))

## Conversion from Decimal

Consider the reverse process that converts a non-negative decimal number of arbitrary size to a string representation in another number system.

### Decimal-to-Binary

The following code converts a decimal number to a binary string.

```python
def decimal_to_binary(decimal):
    binary_str = str(decimal % 2)
    while decimal // 2:
        decimal //= 2
        binary_str = str(decimal % 2) + binary_str
    return binary_str
```

To understand the while loop, consider the same formula {eq}`eq:b2d:3`, where the braces indicate the value of `decimal` at different times:

$$
\begin{aligned} \sum_{i=0}^{k-1} 2^i \cdot b_{i} &=  \left(\sum_{i=0}^{k-2} 2^{i-2} \cdot b_{i-1}\right)\times 2 + b_0 \\
&= \underbrace{(\underbrace{ \dots (\underbrace{(0\times 2 + b_{k-1}) \times 2 + b_{k-2}}_{\text{right before the last iteration} }  )\times 2 \dots + b_1}_{\text{right before the second iteration} })\times 2 + b_0}_{\text{right before the first iteration} }.\end{aligned}
$$

- $b_0$ is the remainder `decimal % 2` right before the first iteration,
- $b_1$ is the remainder `decimal // 2 % 2` right before the second iteration, and
- $b_{k-1}$ is the remainder `decimal // 2 % 2` right before the last iteration.

We can also write a for loop instead of a while loop:

In [ ]:
def decimal_to_binary(decimal):
    binary_str = ""
    num_bits = 1 + (decimal and floor(log2(decimal)))
    for i in range(num_bits):
        binary_str = str(decimal % 2) + binary_str
        decimal //= 2
    return binary_str

In [ ]:
# decimal-to-binary calculator
@interact(decimal="11")
def convert_decimal_to_binary(decimal):
    if not decimal.isdigit():
        print("Not a non-negative integer.")
    else:
        print("binary:", decimal_to_binary(int(decimal)))

::::{exercise} logical and
:label: ex:logical-and

Explain what the expression `1 + (decimal and floor(log2(decimal)))` calculates. In particular, explain the purpose of the logical `and` operation in the expression?

:::{hint}
:class: dropdown

`floor(log2(decimal))` computes $\lfloor \log_2 (\texttt{decimal})\rfloor$, namely, the smallest integer no larger than the logarithm base 2 of some number `decimal`.
- What happen when you run `floor(log2(decimal))` when `decimal` equals `0`?
- How about `(decimal and floor(log2(decimal))` when `decimal` equals `0`? Apply the short-circuit evaluation rule.
- What does `1 + (decimal and floor(log2(decimal)))` return when `decimal` is between $2^{\ell} \leq \texttt{decimal} < 2^{\ell + 1}$?
:::

::::

YOUR ANSWER HERE

### Decimal-to-Undecimal

::::{exercise} decimal-to-undecimal conversion
:label: ex:decimal-to-undecimal

Assign to `undecimal_str` the undecimal string that represents a non-negative integer `decimal` of any size.

::::

In [ ]:
def decimal_to_undecimal(decimal):
    # YOUR CODE HERE
    raise NotImplementedError
    return undecimal_str

In [ ]:
# tests
def test_decimal_to_undecimal(undecimal, decimal):
    undecimal_ = decimal_to_undecimal(decimal)
    correct = isinstance(undecimal, str) and undecimal == undecimal_
    if not correct:
        print(
            f"{decimal} should be represented as the undecimal string {undecimal}, not {undecimal_}."
        )
    assert correct


test_decimal_to_undecimal("X", 10)
test_decimal_to_undecimal("0", 0)
test_decimal_to_undecimal("1752572309X478", 57983478668530)

In [ ]:
# hidden tests

In [ ]:
# undecimal-to-decimal calculator
@interact(decimal="10")
def convert_decimal_to_undecimal(decimal):
    if not decimal.isdigit():
        print("Not a non-negative integer.")
    else:
        print("undecimal:", decimal_to_undecimal(int(decimal)))